In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-5d5656e8-30cf-0415-14cf-2017e1b07bee)


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/sign_language/lcrnet-v2-improved-ppi')
#from lcr_net_ppi_improved import LCRNet_PPI_improved
sys.path.append('/content/drive/MyDrive/sign_language/DOPE')
import gzip
import pickle
import sys, os
import argparse
import os.path as osp
from PIL import Image
import cv2
import numpy as np
import time

import torch
import torch.nn as nn
from torchvision.transforms import ToTensor

from model import dope_resnet50, num_joints
import postprocess

import matplotlib.pyplot as plt

# Load features files PHOENIX

In [ ]:
def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object
def extract(tmp):
  samples = {}
  for s in tmp:
    seq_id = s["name"]
    if seq_id in samples:
        assert samples[seq_id]["name"] == s["name"]
        assert samples[seq_id]["signer"] == s["signer"]
        assert samples[seq_id]["gloss"] == s["gloss"]
        assert samples[seq_id]["text"] == s["text"]
        samples[seq_id]["sign"] = torch.cat(
            [samples[seq_id]["sign"], s["sign"]], axis=1
        )
    else:
        samples[seq_id] = {
            "name": s["name"],
            "signer": s["signer"],
            "gloss": s["gloss"],
            "text": s["text"],
            "sign": s["sign"],
        }
  return samples

## TRAIN

In [ ]:
tmp = load_dataset_file('/content/drive/MyDrive/sign_language/data/phoenix14t.pami0.train')
samples_train = extract(tmp)

## DEV

In [ ]:
tmp = load_dataset_file('/content/drive/MyDrive/sign_language/data/phoenix14t.pami0.dev')
samples_dev = extract(tmp)

## TEST

In [ ]:
tmp = load_dataset_file('/content/drive/MyDrive/sign_language/data/phoenix14t.pami0.test')
samples_test = extract(tmp)

# Pararaph Dataset

In [ ]:
max_length = 440

## Train

In [ ]:
names_per_prefix = {}
for key in samples_train.keys():
  root = key.split('-')[0]
  num = int(key.split('-')[1])
  try:
    names_per_prefix[root].append(num)
  except KeyError:
    names_per_prefix[root] = [num]

In [ ]:
paragraphs_all_train = []
for prefix in names_per_prefix.keys():
  paragraphs = []
  current = names_per_prefix[prefix][0]
  length = samples_train[prefix + '-' + str(current)]['sign'].shape[0]
  current_paragraph = [prefix + '-' + str(current)]
  for indice, num in enumerate(names_per_prefix[prefix]):
    if indice == 0:
      continue
    length += samples_train[prefix + '-' + str(num)]['sign'].shape[0]
    if num - current > 1 or length > max_length:
      paragraphs.append(current_paragraph)
      length = samples_train[prefix + '-' + str(num)]['sign'].shape[0]
      current_paragraph = [prefix + '-' + str(num)]
    else:
      current_paragraph.append(prefix + '-' + str(num))
    current = num
  paragraphs.append(current_paragraph)
  paragraphs_all_train += paragraphs

In [ ]:
len(paragraphs_all_train)

2811

## Test

In [ ]:
names_per_prefix = {}
for key in samples_test.keys():
  root = key.split('-')[0]
  num = int(key.split('-')[1])
  try:
    names_per_prefix[root].append(num)
  except KeyError:
    names_per_prefix[root] = [num]

In [ ]:
paragraphs_all_test = []
for prefix in names_per_prefix.keys():
  paragraphs = []
  current = names_per_prefix[prefix][0]
  length = samples_test[prefix + '-' + str(current)]['sign'].shape[0]
  current_paragraph = [prefix + '-' + str(current)]
  for indice, num in enumerate(names_per_prefix[prefix]):
    if indice == 0:
      continue
    length += samples_test[prefix + '-' + str(num)]['sign'].shape[0]
    if num - current > 1 or length > max_length:
      paragraphs.append(current_paragraph)
      length = samples_test[prefix + '-' + str(num)]['sign'].shape[0]
      current_paragraph = [prefix + '-' + str(num)]
    else:
      current_paragraph.append(prefix + '-' + str(num))
    current = num
  paragraphs.append(current_paragraph)
  paragraphs_all_test += paragraphs

In [ ]:
len(paragraphs_all_test)

452

## Dev

In [ ]:
names_per_prefix = {}
for key in samples_dev.keys():
  root = key.split('-')[0]
  num = int(key.split('-')[1])
  try:
    names_per_prefix[root].append(num)
  except KeyError:
    names_per_prefix[root] = [num]

In [ ]:
paragraphs_all_dev = []
for prefix in names_per_prefix.keys():
  paragraphs = []
  current = names_per_prefix[prefix][0]
  length = samples_dev[prefix + '-' + str(current)]['sign'].shape[0]
  current_paragraph = [prefix + '-' + str(current)]
  for indice, num in enumerate(names_per_prefix[prefix]):
    if indice == 0:
      continue
    length += samples_dev[prefix + '-' + str(num)]['sign'].shape[0]
    if num - current > 1 or length > max_length:
      paragraphs.append(current_paragraph)
      length = samples_dev[prefix + '-' + str(num)]['sign'].shape[0]
      current_paragraph = [prefix + '-' + str(num)]
    else:
      current_paragraph.append(prefix + '-' + str(num))
    current = num
  paragraphs.append(current_paragraph)
  paragraphs_all_dev += paragraphs

In [ ]:
len(paragraphs_all_dev)

403

In [ ]:
samples_train['train/01April_2010_Thursday_heute-6694']

{'gloss': 'LIEB ZUSCHAUER ABEND',
 'name': 'train/01April_2010_Thursday_heute-6694',
 'sign': tensor([[0.3870, 0.0000, 0.4217,  ..., 0.0000, 0.0000, 0.0000],
         [0.3128, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.5446, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.5000, 0.2338, 1.1811,  ..., 0.0000, 0.0000, 0.0000],
         [0.9756, 0.0178, 0.3846,  ..., 0.0000, 0.0000, 0.0000],
         [0.3156, 0.0715, 0.0034,  ..., 0.0000, 0.0000, 0.0000]]),
 'signer': 'Signer04',
 'text': 'liebe zuschauer guten abend .'}

In [ ]:
paragraphs_all_train[1]

['train/11August_2010_Wednesday_tagesschau-4',
 'train/11August_2010_Wednesday_tagesschau-5']

# Generate data paragraphs

In [ ]:
def save(object, filename, protocol = 0):
        """Saves a compressed object to disk
        """
        file = gzip.GzipFile(filename, 'wb')
        file.write(pickle.dumps(object, protocol))
        file.close()

## Train

In [ ]:
paragraphs_train = {}
for names in paragraphs_all_train:
  data = samples_train[names[0]]
  gloss, text, sign, signer = data['gloss'], data['text'], data['sign'], data['signer']
  for i, name in enumerate(names):
    if i == 0:
      continue
    data = samples_train[name]
    gloss += "  " + data['gloss']
    text += " " + data['text']
    sign = torch.cat([sign, data['sign']])
  paragraphs_train[names[0]] = {'gloss': gloss, 'name': names[0], 'sign': sign,
                            'signer': signer, 'text': text}

In [ ]:
len(paragraphs_train)

2811

In [ ]:
save(paragraphs_train, '/content/drive/MyDrive/sign_language/data/phoenix14t.pami0.trainparagraph')

## Test

In [ ]:
paragraphs_test = {}
for names in paragraphs_all_test:
  data = samples_test[names[0]]
  gloss, text, sign, signer = data['gloss'], data['text'], data['sign'], data['signer']
  for i, name in enumerate(names):
    if i == 0:
      continue
    data = samples_test[name]
    gloss += "  " + data['gloss']
    text += " " + data['text']
    sign = torch.cat([sign, data['sign']])
  paragraphs_test[names[0]] = {'gloss': gloss, 'name': names[0], 'sign': sign,
                            'signer': signer, 'text': text}

In [ ]:
len(paragraphs_all_test)

452

In [ ]:
len(paragraphs_test)

452

In [ ]:
save(paragraphs_test, '/content/drive/MyDrive/sign_language/data/phoenix14t.pami0.testparagraph')

## Dev

In [ ]:
paragraphs_dev = {}
for names in paragraphs_all_dev:
  data = samples_dev[names[0]]
  gloss, text, sign, signer = data['gloss'], data['text'], data['sign'], data['signer']
  for i, name in enumerate(names):
    if i == 0:
      continue
    data = samples_dev[name]
    gloss += "  " + data['gloss']
    text += " " + data['text']
    sign = torch.cat([sign, data['sign']])
  paragraphs_dev[names[0]] = {'gloss': gloss, 'name': names[0], 'sign': sign,
                            'signer': signer, 'text': text}

In [ ]:
len(paragraphs_dev)

403

In [ ]:
len(paragraphs_all_dev)

403

In [ ]:
save(paragraphs_dev, '/content/drive/MyDrive/sign_language/data/phoenix14t.pami0.devparagraph')